In [1]:
import os, sys
import yaml

cwd = os.getcwd()
sys.path.append(os.path.dirname(cwd))
from utils import models_generator as models_gen
from utils import models_discriminator as models_dis
from utils import models_cyclegan as models_gan
from utils import preprocessing as pre
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import sklearn
import seaborn as sns

In [5]:
# Load config

names = ["model1", "model2", "model3"]
file = "../config/Cycle_gan/config.yaml"
with open(file, 'r') as file_descriptor:
    config = yaml.load(file_descriptor, Loader=yaml.FullLoader)
model_input = tf.keras.layers.Input(shape=(512,21))
model_gen = models_gen.Generator_res(config['CycleGan']['Generator'], vocab = 21, name = "therm")
output = model_gen(model_input)
model_gen.summary()

Model: "therm"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
thermdown_0 (Conv1D)         multiple                  12160     
_________________________________________________________________
thermdown_1 (Conv1D)         multiple                  36928     
_________________________________________________________________
thermres_0 (ResModPreAct)    multiple                  78016     
_________________________________________________________________
thermres_1 (ResModPreAct)    multiple                  78016     
_________________________________________________________________
thermres_2 (ResModPreAct)    multiple                  229760    
_________________________________________________________________
thermres_3 (ResModPreAct)    multiple                  311680    
_________________________________________________________________
thermres_4 (ResModPreAct)    multiple                  918272

In [3]:
config["Data"]["base_dir"] = "../data"
config["Data"]["train_dir"] = "Combined_data/published/Groups_5/RECORDS_REG_GLOBAL/train/*.tfrecord"
config["Data"]["val_dir"] = "Combined_data/published/Groups_5/RECORDS_REG_GLOBAL/val/*.tfrecord"

_, data_val = pre.load_data(config["Data"])
data_val = [(i[0], i[1]) for i in data_val.batch(1).as_numpy_iterator()]

for seq, val in data_val:
    ten = model1(seq)
    break
print(ten)

tf.Tensor(
[[[4.9101859e-01 2.7956925e-02 2.1174178e-03 ... 1.0147896e-01
   4.7793142e-03 5.4996848e-02]
  [2.4855126e-01 1.3730978e-03 4.1091954e-03 ... 4.9080228e-04
   9.3110884e-03 2.4887645e-01]
  [2.8153718e-03 2.4949366e-02 6.4874858e-01 ... 4.9024668e-02
   1.2066399e-03 2.5845429e-05]
  ...
  [8.0125566e-05 7.5826648e-04 1.0030077e-02 ... 1.3042762e-04
   2.3690898e-05 1.3959467e-04]
  [1.6432139e-04 1.2928373e-03 1.6995370e-04 ... 1.2760801e-04
   1.4927003e-02 1.4926915e-04]
  [5.7703524e-04 6.3740313e-02 1.7024770e-04 ... 8.1554577e-03
   6.3444309e-02 1.4737853e-03]]], shape=(1, 512, 21), dtype=float32)


In [5]:
model1.get_layer(index = 8).trainable_weights

[<tf.Variable 'thermattentiongamma:0' shape=() dtype=float32, numpy=1.0>,
 <tf.Variable 'therm/thermattention/thermattentionsn_0/kernel:0' shape=(256, 256) dtype=float32, numpy=
 array([[-0.00999945, -0.00957488, -0.07804994, ...,  0.05519979,
         -0.04628043,  0.0620893 ],
        [-0.02211439,  0.02783603,  0.04493883, ..., -0.05187117,
          0.07513788,  0.08473777],
        [ 0.09375375, -0.01943791,  0.03837272, ...,  0.08380274,
          0.05084942, -0.05788479],
        ...,
        [ 0.05704554, -0.0920013 ,  0.0775527 , ..., -0.03908756,
          0.01666692,  0.05242445],
        [-0.0251561 ,  0.05549287, -0.04594836, ..., -0.05807462,
         -0.04052334,  0.02076911],
        [-0.07072366,  0.00090421, -0.00312621, ...,  0.03416356,
          0.04619438, -0.05937019]], dtype=float32)>,
 <tf.Variable 'therm/thermattention/thermattentionsn_0/bias:0' shape=(256,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
     

In [9]:
print(output)

KerasTensor(type_spec=TensorSpec(shape=(None, 64, 1), dtype=tf.float32, name=None), name='dis_therm/dis_therm_conv_out/Sigmoid:0', description="created by layer 'dis_therm'")


In [3]:
model1.save_weights("test1.h5", overwrite = True)

In [4]:
model1.load_weights("test1.h5", by_name = True)

In [8]:
file = "../config/Cycle_gan/config.yaml"
with open(file, 'r') as file_descriptor:
    config = yaml.load(file_descriptor, Loader=yaml.FullLoader)
model_input = tf.keras.layers.Input(shape=(512,21))
model1 = models_dis.Discriminator(config['CycleGan']['Discriminator'], vocab = 21, name = "dis_therm")
output = model1(model_input)

In [10]:
model1.summary()

Model: "dis_therm"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dis_thermres_0 (ResModPreAct multiple                  87424     
_________________________________________________________________
dis_thermres_1 (ResModPreAct multiple                  78016     
_________________________________________________________________
dis_thermres_2 (ResModPreAct multiple                  229760    
_________________________________________________________________
dis_thermres_3 (ResModPreAct multiple                  459264    
_________________________________________________________________
dis_thermres_4 (ResModPreAct multiple                  918272    
_________________________________________________________________
dis_thermres_5 (ResModPreAct multiple                  1836032   
_________________________________________________________________
dis_therm_attention (SelfAtt multiple                  66

In [2]:
file = "../config/Cycle_gan/config.yaml"
with open(file, 'r') as file_descriptor:
    config = yaml.load(file_descriptor, Loader=yaml.FullLoader)
models_gan.CycleGan(config, name = "gan")

Model: "Generator_thermo"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Generator_thermo_down_0 (Con multiple                  12160     
_________________________________________________________________
Generator_thermo_down_1 (Con multiple                  36928     
_________________________________________________________________
Generator_thermo_res_0 (ResM multiple                  78016     
_________________________________________________________________
Generator_thermo_res_1 (ResM multiple                  78016     
_________________________________________________________________
Generator_thermo_res_2 (ResM multiple                  229760    
_________________________________________________________________
Generator_thermo_res_3 (ResM multiple                  311680    
_________________________________________________________________
Generator_thermo_res_4 (ResM multiple             